In [ ]:
# Generic Imports
import numpy as np 
from PIL import Image
import scipy.io as sio 
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt 



In [ ]:

# ── Hyperparameters ───────────────────────────────────────────────────────────
BETA_0    = 0.5
BETA_F    = 10.0
BETA_RATE = 1.075
I0        = 4
I1        = 30
EPS       = 0.5
OBJ_RESIZE = (256, 256)

# ── Provided utilities ────────────────────────────────────────────────────────
def load_and_preprocess_images(img_path1, img_path2, mat_path1, mat_path2):
    """Load images and scale keypoints to obj_resize."""
    return img1, img2, kpts1, kpts2

def delaunay_triangulation(kpt):
    """Build adjacency matrix from Delaunay triangulation."""
    return A

def visualize_matching_full(img1, img2, kpts1, kpts2, adj1, adj2, matching):
    """Visualize matching: yellow edges, green correct, red incorrect."""
    pass

In [ ]:
# ── TO IMPLEMENT ──────────────────────────────────────────────────────────────
def sinkhorn(M, max_iter=30, eps=1e-3):
    """
    Alternate row/column normalization until M is doubly stochastic.

    Args:
        M        : (m×n) non-negative matrix
        max_iter : maximum iterations
        eps      : convergence threshold on ||M - M_old||₁

    Returns:
        M : doubly stochastic matrix (rows and columns sum to 1)

    Hints:
        - Normalize rows:    M = M / M.sum(axis=1, keepdims=True)
        - Normalize columns: M = M / M.sum(axis=0, keepdims=True)
        - Add 1e-10 in the denominator to avoid division by zero
        - Stop early if ||M - M_old||₁ < eps
    """
    
    for _ in range(max_iter):
        M_old = M.copy()
        # Normalizamos
        M = M/ (M.sum(axis=1, keepdims=True)) # Filas
        M = M/ (M.sum(axis=0, keepdims=True)) # Columnas
        if np.abs(M-M_old).sum() < eps:
            break

    return M

In [ ]:
# ── TO IMPLEMENT ──────────────────────────────────────────────────────────────
def softassign(X_adj, Y_adj,
               beta0=BETA_0, beta_f=BETA_F, beta_rate=BETA_RATE,
               i0=I0, i1=I1, eps=EPS):
    """
    Graph matching by maximizing structural rectangles (Gold & Rangarajan 1996).

    Args:
        X_adj     : (m×m) adjacency matrix of G1
        Y_adj     : (n×n) adjacency matrix of G2

    Returns:
        M : (m×n) soft matching matrix

    Hints:
        - Initialize: M = ones(m,n) + 0.1*rand(m,n), then Sinkhorn
        - Outer loop: while beta < beta_f, multiply beta by beta_rate at the end
        - Inner loop (i0 iterations):
            · Q = X_adj @ M @ Y_adj
            · M = exp(beta * Q)
            · M = sinkhorn(M)
            · break if ||M - M_old||₁ < eps
    """
    
    n = X_adj
    M = np.ones((n, n))
    M = sinkhorn(M, max_iter=11) # Hacer estocástica
    # Matriz de Costes
    while beta < beta_f:
        for _ in range(i0):
            M_old = M.copy()
            Q = X_adj @ M @ Y_adj
            M = np.exp(beta*Q)
            M = sinkhorn(M, max_iter=11)
            
            if np.abs(M - M_old).sum() < eps:
                break 
        
        beta*= beta_rate
        
    '''
    for a in range(n):
        for i in range(n):
            for b in range(n):
                for j in range(n):
                    Q[a, i] += X_adj[a, b] * M[b, j] * Y_adj[i, j]
                    '''
    return M


In [ ]:
# ── TO IMPLEMENT ──────────────────────────────────────────────────────────────

def cleanup(M_soft):
    """
    Convert soft matching to binary via greedy assignment.

    Args:
        M_soft : (m×n) soft matching matrix

    Returns:
        M_binary : (m×n) binary matching matrix

    Hints:
        - Repeatedly find argmax of M_work
        - Assign that entry, then set its row and column to -inf
    """
    len_x = M_soft.size[0]
    len_y = M_soft.size[1]

    M_binary = np.zeros(len_x, len_y)
    M_work = M_soft.copy() 
    
    for k in range(min(len_x, len_y)):
        a, i = np.argmax(M_work, axis=1)
    
    
    return M_binary